In [1]:
import numpy as np
import h5py
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file = h5py.File('/content/drive/MyDrive/BCI/MI_3.mat', 'r')
X = np.array(file['X'])  # 原 shape: (576, 3, 500)
X = X.transpose(2, 1, 0)  # → (576, 500, 3)
X = X.reshape(-1, 500)  # (576 × 3, 500) → (1728, 500)

y = np.array(file['y']).flatten()  # (576,)
# 標籤也要複製三次，讓每個 channel 對應正確 label
y = np.repeat(y, 3)  # 576 → 1728
print(f'✅ X shape: {X.shape}, y shape: {y.shape}')

# === 拆分資料：80% 訓練+驗證，20% 測試 ===
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# === Cross-validation 設定 ===
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_acc_list = []
val_acc_list = []
class_correct = defaultdict(int)
class_total = defaultdict(int)

fold = 1
for train_idx, val_idx in skf.split(X_trainval, y_trainval):
    X_train, X_val = X_trainval[train_idx], X_trainval[val_idx]
    y_train, y_val = y_trainval[train_idx], y_trainval[val_idx]

    # 特徵標準化
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    # 建立 XGBoost 模型
    clf = XGBClassifier(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        eval_metric='logloss',
        random_state=42
    )
    clf.fit(X_train, y_train)

    y_train_pred = clf.predict(X_train)
    y_val_pred = clf.predict(X_val)

    train_acc = accuracy_score(y_train, y_train_pred)
    val_acc = accuracy_score(y_val, y_val_pred)

    print(f'📘 Fold {fold}: Train Acc = {train_acc:.4f} | Val Acc = {val_acc:.4f}')
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    fold += 1

    # 每類別的驗證正確率
    for true, pred in zip(y_val, y_val_pred):
        class_total[true] += 1
        if true == pred:
            class_correct[true] += 1

# === 在全部 trainval 上重新訓練一次 ===
scaler_final = StandardScaler()
X_trainval_scaled = scaler_final.fit_transform(X_trainval)
X_test_scaled = scaler_final.transform(X_test)

clf_final = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
clf_final.fit(X_trainval_scaled, y_trainval)

y_test_pred = clf_final.predict(X_test_scaled)
test_acc = accuracy_score(y_test, y_test_pred)

# === 輸出結果 ===
print("\n✅ Cross-Validation Summary")
print(f'📊 平均 Train Acc: {np.mean(train_acc_list):.4f}')
print(f'🧪 平均 Val Acc: {np.mean(val_acc_list):.4f}')

print("\n🎯 每類別驗證集正確率：")
label_names = ['Left Hand', 'Right Hand']
for i in [0, 1]:
    acc = class_correct[i] / class_total[i] if class_total[i] > 0 else 0
    print(f'📡 {label_names[i]} Accuracy: {acc:.4f} ({class_correct[i]}/{class_total[i]})')

print(f'\n🚀 測試集準確率: {test_acc:.4f}')
print("\n📋 測試集分類報告:")
print(classification_report(y_test, y_test_pred, target_names=label_names))

✅ X shape: (1728, 500), y shape: (1728,)
📘 Fold 1: Train Acc = 1.0000 | Val Acc = 0.7762
📘 Fold 2: Train Acc = 1.0000 | Val Acc = 0.8087
📘 Fold 3: Train Acc = 1.0000 | Val Acc = 0.7971
📘 Fold 4: Train Acc = 1.0000 | Val Acc = 0.7862
📘 Fold 5: Train Acc = 1.0000 | Val Acc = 0.7536


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:16:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ Cross-Validation Summary
📊 平均 Train Acc: 1.0000
🧪 平均 Val Acc: 0.7844

🎯 每類別驗證集正確率：
📡 Left Hand Accuracy: 0.7742 (535/691)
📡 Right Hand Accuracy: 0.7945 (549/691)

🚀 測試集準確率: 0.8555

📋 測試集分類報告:
              precision    recall  f1-score   support

   Left Hand       0.88      0.82      0.85       173
  Right Hand       0.83      0.89      0.86       173

    accuracy                           0.86       346
   macro avg       0.86      0.86      0.86       346
weighted avg       0.86      0.86      0.86       346

